In [1]:
import csv
import os
import sys
import numpy as np
import SimpleITK as sitk
from scipy.ndimage import zoom

def readCsv(csvfname):
    # read csv to list of lists
    with open(csvfname, 'r') as csvf:
        reader = csv.reader(csvf)
        csvlines = list(reader)
    return csvlines

def writeCsv(csfname,rows):
    # write csv from list of lists
    with open(csfname, 'w', newline='') as csvf:
        filewriter = csv.writer(csvf)
        filewriter.writerows(rows)
        
def readMhd(filename):
    # read mhd/raw image
    itkimage = sitk.ReadImage(filename)
    scan = sitk.GetArrayFromImage(itkimage) #3D image
    spacing = itkimage.GetSpacing() #voxelsize
    origin = itkimage.GetOrigin() #world coordinates of origin
    transfmat = itkimage.GetDirection() #3D rotation matrix
    return scan,spacing,origin,transfmat

def writeMhd(filename,scan,spacing,origin,transfmat):
    # write mhd/raw image
    itkim = sitk.GetImageFromArray(scan, isVector=False) #3D image
    itkim.SetSpacing(spacing) #voxelsize
    itkim.SetOrigin(origin) #world coordinates of origin
    itkim.SetDirection(transfmat) #3D rotation matrix
    sitk.WriteImage(itkim, filename, False)    

def getImgWorldTransfMats(spacing,transfmat):
    # calc image to world to image transformation matrixes
    transfmat = np.array([transfmat[0:3],transfmat[3:6],transfmat[6:9]])
    for d in range(3):
        transfmat[0:3,d] = transfmat[0:3,d]*spacing[d]
    transfmat_toworld = transfmat #image to world coordinates conversion matrix
    transfmat_toimg = np.linalg.inv(transfmat) #world to image coordinates conversion matrix
    
    return transfmat_toimg,transfmat_toworld

def convertToImgCoord(xyz,origin,transfmat_toimg):
    # convert world to image coordinates
    xyz = xyz - origin
    xyz = np.round(np.matmul(transfmat_toimg,xyz))    
    return xyz
    
def convertToWorldCoord(xyz,origin,transfmat_toworld):
    # convert image to world coordinates
    xyz = np.matmul(transfmat_toworld,xyz)
    xyz = xyz + origin
    return xyz

def extractCube(scan,spacing,xyz,cube_size=64,cube_size_mm=51):
    # Extract cube of cube_size^3 voxels and world dimensions of cube_size_mm^3 mm from scan at image coordinates xyz
    xyz = np.array([xyz[i] for i in [2,1,0]],np.int)
    spacing = np.array([spacing[i] for i in [2,1,0]])
    scan_halfcube_size = np.array(cube_size_mm/spacing/2,np.int)
    if np.any(xyz<scan_halfcube_size) or np.any(xyz+scan_halfcube_size>scan.shape): # check if padding is necessary
        maxsize = max(scan_halfcube_size)
        scan = np.pad(scan,((maxsize,maxsize,)),'constant',constant_values=0)
        xyz = xyz+maxsize
    
    scancube = scan[xyz[0]-scan_halfcube_size[0]:xyz[0]+scan_halfcube_size[0], # extract cube from scan at xyz
                    xyz[1]-scan_halfcube_size[1]:xyz[1]+scan_halfcube_size[1],
                    xyz[2]-scan_halfcube_size[2]:xyz[2]+scan_halfcube_size[2]]

    sh = scancube.shape
    scancube = zoom(scancube,(cube_size/sh[0],cube_size/sh[1],cube_size/sh[2]),order=2) #resample for cube_size
    
    return scancube

    
    

In [2]:
import SimpleITK as sitk
import numpy as np
import pandas
import os
import glob
import tensorflow as tf
import gc
from tensorflow import keras
from tensorflow.keras import layers
import random
from scipy import ndimage
import matplotlib.pyplot as plt
from skimage import color
from numpy import newaxis
import csv
import sys
from scipy.ndimage import zoom

In [3]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Model
from keras.layers import *
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
from keras.preprocessing import image
from tensorflow.keras.optimizers import Adam
adam = Adam(learning_rate=0.01)

In [4]:
X= []
y= []

dataset = "basedata/"
file_list = glob.glob(dataset + '*.mhd')


for file_path in file_list: 
    base = os.path.basename(file_path)
    lnd = int(os.path.splitext(base)[0][5:])
    [scan,spacing,origin,transfmat] =  readMhd(file_path)
    
    csvlines = readCsv("trainset_csv/trainNodules_gt.csv")
    header = csvlines[0]
    nodules = csvlines[1:]
    itr=0
    flag=0
    max_agreement=0
    for n in nodules:
        if int(n[header.index('LNDbID')])==lnd:
            if(itr==0 or int(n[header.index('AgrLevel')])>max_agreement):
                ctr = np.array([float(n[header.index('x')]), float(n[header.index('y')]), float(n[header.index('z')])])
                y.append(int(n[header.index('Nodule')]))
#                 print(lnd,int(n[header.index('Nodule')]))
                max_agreement=int(n[header.index('AgrLevel')])
                flag=1
                break
    
#   Convert coordinates to image
    if(flag==1):
        transfmat_toimg,transfmat_toworld = getImgWorldTransfMats(spacing,transfmat)
        ctr = convertToImgCoord(ctr,origin,transfmat_toimg)
        scan_cube = extractCube(scan,spacing,ctr)
        X.append(scan_cube[int(scan_cube.shape[0]/2),:,:])
#         plt.imshow(scan_cube[int(scan_cube.shape[0]/2),:,:])
    

<ipython-input-1-0c370ad15cb4>:62: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  xyz = np.array([xyz[i] for i in [2,1,0]],np.int)
<ipython-input-1-0c370ad15cb4>:64: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in Num

In [6]:
# Get images as a 4,096 feature set
SAMPLE_SIZE = len(y)
data = np.array(X).flatten().reshape(SAMPLE_SIZE, 64*64) # pixel-features

# Turn X and y into numpy arrays
X = np.array(X).reshape(-1, 64, 64) # images
y = np.array(y) # target

print("Features, X shape: ", X.shape)
print("Target, y shape: ", y.shape)
print("Data shape: ", data.shape)

Features, X shape:  (229, 64, 64)
Target, y shape:  (229,)
Data shape:  (229, 4096)


In [7]:
import pickle

#Saves us from having to regenerate our data by saving our data
pickle_out = open("X.pickle", "wb")
pickle.dump(X, pickle_out)
pickle_out.close()

pickle_out = open("y.pickle", "wb")
pickle.dump(y, pickle_out)
pickle_out.close()

pickle_out = open("data.pickle", "wb")
pickle.dump(data, pickle_out)
pickle_out.close()